<a href="https://colab.research.google.com/github/fasterinnerlooper/Notebooks/blob/main/training/training_with_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install wandb

In [28]:
import wandb
wandb.login()
wandb.init(project="codeBERTa-on-csharp")

True

In [3]:
%pip install numpy transformers datasets
%pip install transformers[torch]
!pip install accelerate -U

In [4]:
# This script needs these libraries to be installed:
#   numpy, transformers, datasets

import wandb

import os
import numpy as np
from datasets import load_dataset
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

def compute_metrics(eval_pred):
    logits, gt = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": np.mean(predictions == gt)}

model_name = "huggingface/CodeBERTa-small-v1"

# download prepare the data
#dataset = load_dataset("code_search_net","javascript")
dataset = load_dataset("microsoft/LCC_csharp")

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

In [6]:
dataset = dataset.map(lambda x: tokenizer(x['context'], truncation=True, padding='max_length'), batched=True, num_proc=4, remove_columns=dataset['train'].column_names)

In [8]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [31]:
from huggingface_hub import notebook_login
notebook_login()

In [24]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(300))

block_size = 250

def group_texts(examples):
    # Concatenate all texts.
    #concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    concatenated_examples = examples
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

small_train_dataset = small_train_dataset.map(group_texts, truncation=True, batched=True, num_proc=4)
small_eval_dataset = small_eval_dataset.map(group_texts, truncation=True, batched=True, num_proc=4)

TypeError: ignored

In [33]:
# download the model
model = AutoModelForMaskedLM.from_pretrained(model_name)

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="training-with-wandb"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

# pass "wandb" to the 'report_to' parameter to turn on wandb logging
training_args = TrainingArguments(
    output_dir='models',
    report_to="wandb",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True
)

# define the trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
)

Some weights of the model checkpoint at huggingface/CodeBERTa-small-v1 were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
trainer.train()

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

Epoch,Training Loss,Validation Loss
1,No log,3.519433
2,No log,3.158688
3,No log,3.115144


eval/loss,▁█▂▁
eval/runtime,█▁▃▂
eval/samples_per_second,▁█▆▇
eval/steps_per_second,▁█▆▇
train/epoch,█▁▅██
train/global_step,█▁▅██
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁


In [35]:
import math

wandb.init(project="codeBERTa-on-csharp")

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 21.59
